In [1]:
# AOD historical data close to Fort Worth 
# Latitude = 32.759154
# Longitude = -97.342332

In [1]:
from netCDF4 import Dataset
import numpy as np
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

In [2]:
file_id = Dataset("/home/prabu/Research/GOES_16_data/OR_ABI-L2-AODC-M6_G16_s20211400101152_e20211400103525_c20211400105138.nc")

In [3]:
# Finding Latitudes and Longitudes of AOD data

proj_info = file_id.variables['goes_imager_projection']
lon_origin = proj_info.longitude_of_projection_origin
H = proj_info.perspective_point_height+proj_info.semi_major_axis
r_eq = proj_info.semi_major_axis
r_pol = proj_info.semi_minor_axis

lat_rad_1d = file_id.variables['x'][:]
lon_rad_1d = file_id.variables['y'][:]

#Create meshgrid filled with radian angles
lat_rad,lon_rad = np.meshgrid(lat_rad_1d,lon_rad_1d)
#lat/lon calculus routine from satellite radian angle vectors
lambda_0 = (lon_origin*np.pi)/180.0

a_var = np.power(np.sin(lat_rad),2.0) + (np.power(np.cos(lat_rad),2.0)*(np.power(np.cos(lon_rad),2.0)+(((r_eq*r_eq)/(r_pol*r_pol))*np.power(np.sin(lon_rad),2.0))))
b_var = -2.0*H*np.cos(lat_rad)*np.cos(lon_rad)
c_var = (H**2.0)-(r_eq**2.0)
    
r_s = (-1.0*b_var - np.sqrt((b_var**2)-(4.0*a_var*c_var)))/(2.0*a_var)
    
s_x = r_s*np.cos(lat_rad)*np.cos(lon_rad)
s_y = - r_s*np.sin(lat_rad)
s_z = r_s*np.cos(lat_rad)*np.sin(lon_rad)
    
Lat = (180.0/np.pi)*(np.arctan(((r_eq*r_eq)/(r_pol*r_pol))*((s_z/np.sqrt(((H-s_x)*(H-s_x))+(s_y*s_y))))))
Lon = (lambda_0 - np.arctan(s_y/(H-s_x)))*(180.0/np.pi)

/tmp/ipykernel_3662/1222391679.py:19: RuntimeWarning: invalid value encountered in sqrt
  r_s = (-1.0*b_var - np.sqrt((b_var**2)-(4.0*a_var*c_var)))/(2.0*a_var)
/tmp/ipykernel_3662/1222391679.py:25: RuntimeWarning: invalid value encountered in sqrt
  Lat = (180.0/np.pi)*(np.arctan(((r_eq*r_eq)/(r_pol*r_pol))*((s_z/np.sqrt(((H-s_x)*(H-s_x))+(s_y*s_y))))))


In [57]:
# Finding index of closest location to Fort Worth from 2D array of Latitudes and Longitudes 

X_FW = -97.342332   # Fort Worth Longitude
Y_FW = 32.759154    # Fort Worth Latitudes 

x = Lon[:][:]
y = Lat[:][:]

delt_x = abs(X_FW - x)
delt_y = abs(Y_FW - y)
dis = (((delt_x)**(2))+((delt_y)**(2)))**(.5)    # Distance between Fort Worth and all the Latitudes and Longitudes

index = np.where(dis == np.min(dis))  # index of minimum distance
print(index)

(array([647]), array([835]))


In [59]:
# AOD data for Latitudes and Longitudes in 2D array
AOD_data = file_id.variables['AOD'][:,:]

AOD_FW = AOD_data[647][835]    # AOD value near to Fort Worth 
print(AOD_FW)

0.27103195
